In [1]:
!pip install fastai --upgrade


Requirement already up-to-date: fastai in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (1.0.57)


In [2]:
from fastai import *
from fastai.tabular import *

In [3]:
input_path = './'
train_df = pd.read_csv(f'{input_path}train.csv')
test_df = pd.read_csv(f'{input_path}test.csv')

In [4]:
for df in [train_df, test_df]:
    df['Title'] = df['Name'].str.split(',').str[1].str.split(' ').str[1]
    df['Deck'] = df['Cabin'].str[0]

In [5]:
all_df = pd.concat([train_df, test_df], sort=False)
mean_age_by_title = all_df.groupby('Title').mean()['Age']

In [6]:
for df in [train_df, test_df]:
    for title, age in mean_age_by_title.iteritems():
        df.loc[df['Age'].isnull() & (df['Title'] == title), 'Age'] = age

In [8]:
mean_fare_by_title = all_df.groupby('Title').mean()['Fare']

In [9]:
for df in [train_df, test_df]:
    for title, fare in mean_fare_by_title.iteritems():
        df.loc[df['Fare'].isnull() & (df['Title'] == title), 'Fare'] = fare

In [10]:
dep_var = 'Survived'
cat_names = ['Pclass', 'Sex', 'Embarked', 'Title', 'Deck']
cont_names = ['Age', 'Fare', 'SibSp', 'Parch']
procs = [FillMissing, Categorify, Normalize]

test = TabularList.from_df(test_df, cat_names=cat_names, cont_names=cont_names, procs=procs)
data = (TabularList.from_df(train_df, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(0,200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test, label=0)
                           .databunch())

In [11]:
np.random.seed(101)
learn = tabular_learner(data, layers=[60, 20], metrics=accuracy)
learn.fit(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.619129,0.664838,0.660000,00:08
1,0.547597,0.613899,0.680000,00:00
2,0.507134,0.522237,0.785000,00:00
3,0.474761,0.457994,0.810000,00:00
4,0.452457,0.437163,0.820000,00:00


In [12]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)

In [13]:
sub_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': labels})
sub_df.to_csv('submission.csv', index=False)

In [14]:
sub_df.tail()

,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,1


In [17]:
create_download_link(sub_df)

In [16]:
from IPython.display import HTML
import base64

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = f'<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    return HTML(html)